In [ ]:
# Install gdown if not already installed
!pip install gdown

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score
import joblib


In [ ]:
# Download the predictions.csv file from the zip
!gdown --id 16XhMwDZ3ZcxNkts6jo3IjPxU-Z-cSNLj -O predictions_distillbert.zip

# Extract the zip file
import zipfile
with zipfile.ZipFile("predictions_distillbert.zip", 'r') as zip_ref:
    zip_ref.extractall()


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16XhMwDZ3ZcxNkts6jo3IjPxU-Z-cSNLj
To: /content/predictions_distillbert.zip
100% 3.59M/3.59M [00:00<00:00, 92.2MB/s]


In [ ]:
# Download the predictions file for logistic regression from the zip
!gdown --id 1b7PEr5VpDtHAD10VuYhP3KrLCtqMqoiF -O predictions_logistic_regression.zip

# Extract the zip file
import zipfile
with zipfile.ZipFile("predictions_logistic_regression.zip", 'r') as zip_ref:
    zip_ref.extractall()


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1b7PEr5VpDtHAD10VuYhP3KrLCtqMqoiF
To: /content/predictions_logistic_regression.zip
100% 3.46M/3.46M [00:00<00:00, 131MB/s]


In [ ]:
# Download the ZIP file from Google Drive
!gdown --id 1LDmkBpIjEJy3rMghZEF3evM2Imt0wu9g -O test_processed.zip

# Unzip the file
import shutil
shutil.unpack_archive('test_processed.zip')

print("File unzipped. Ready to load.")


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LDmkBpIjEJy3rMghZEF3evM2Imt0wu9g
To: /content/test_processed.zip
100% 13.3M/13.3M [00:00<00:00, 43.6MB/s]
File unzipped. Ready to load.


In [ ]:
# Load the predictions
predictions_df = pd.read_csv("predictions_distillbert.csv")
print(predictions_df.head())


      toxic  severe_toxic   obscene    threat    insult  identity_hate
0  0.000283      0.000098  0.000207  0.000131  0.000159       0.000115
1  0.468506      0.000438  0.005158  0.002016  0.007613       0.001063
2  0.476459      0.000736  0.187429  0.001025  0.050380       0.001523
3  0.000269      0.000049  0.000184  0.000063  0.000108       0.000069
4  0.000235      0.000063  0.000175  0.000075  0.000114       0.000073


In [ ]:
X_test = joblib.load("X_test.joblib")
y_test = joblib.load("y_test.joblib")

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])


In [ ]:
roc_auc_scores = {}
for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    score = roc_auc_score(y_test_df[label], predictions_df[label])
    roc_auc_scores[label] = score
    print(f"ROC AUC for {label}: {score}")
print(f"Mean ROC AUC: {np.mean(list(roc_auc_scores.values()))}")


ROC AUC for toxic: 0.9729149391929242
ROC AUC for severe_toxic: 0.9896442045116097
ROC AUC for obscene: 0.9806681704851719
ROC AUC for threat: 0.9951983810729182
ROC AUC for insult: 0.9808542745502147
ROC AUC for identity_hate: 0.9896246213153168
Mean ROC AUC: 0.9848174318546925


In [ ]:
import json

# Save the results in JSON format
with open('roc_auc_scores_distillbert.json', 'w') as f:
    json.dump(roc_auc_scores, f, indent=4)

print("Results saved to roc_auc_scores_distillbert.json")


Results saved to roc_auc_scores_distillbert.json


In [ ]:
threshold = 0.5
binary_predictions = (predictions_df.values > threshold).astype(int)
report_dict = classification_report(
    y_test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    binary_predictions,
    target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
    output_dict=True
)
with open('classification_report_distillbert.json', 'w') as f:
    json.dump(report_dict, f, indent=4)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print(classification_report(
    y_test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    binary_predictions,
    target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
))


               precision    recall  f1-score   support

        toxic       0.55      0.91      0.68      6090
 severe_toxic       0.41      0.37      0.39       367
      obscene       0.61      0.81      0.69      3691
       threat       0.56      0.49      0.52       211
       insult       0.69      0.74      0.71      3427
identity_hate       0.70      0.56      0.62       712

    micro avg       0.59      0.81      0.68     14498
    macro avg       0.58      0.65      0.60     14498
 weighted avg       0.60      0.81      0.68     14498
  samples avg       0.08      0.08      0.08     14498



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_pred_proba = pd.read_csv("predictions_logistic_regression.csv")
y_pred_proba.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.050895,0.002917,0.012291,0.001091,0.049769,0.012184
1,0.948713,0.013770,0.319479,0.049329,0.417398,0.015253
2,0.229789,0.015104,0.096291,0.002132,0.061374,0.049543
3,0.028072,0.005561,0.032281,0.001477,0.017489,0.008180
4,0.017987,0.001589,0.017322,0.000279,0.008532,0.000414


In [ ]:
threshold = 0.5

y_pred_binary = (y_pred_proba.values > threshold).astype(int)


roc_auc_scores_logistic = {}
for label in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    score = roc_auc_score(y_test_df[label], y_pred_proba[label])
    roc_auc_scores_logistic[label] = score
    print(f"ROC AUC for {label}: {score}")
print(f"Mean ROC AUC: {np.mean(list(roc_auc_scores_logistic.values()))}")



ROC AUC for toxic: 0.9542307122592656
ROC AUC for severe_toxic: 0.9803977573669523
ROC AUC for obscene: 0.9691850146205508
ROC AUC for threat: 0.9816574143558929
ROC AUC for insult: 0.9622717941029407
ROC AUC for identity_hate: 0.9716843955981114
Mean ROC AUC: 0.9699045147172857


In [ ]:
import json
with open('roc_auc_scores_logistic.json', 'w') as f:
    json.dump(roc_auc_scores_logistic, f, indent=4)

print("Results saved to roc_auc_scores_logistic.json")

Results saved to roc_auc_scores_logistic.json


In [ ]:
report_dict_logistic = classification_report(
    y_test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    y_pred_binary,
    target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
    output_dict=True
)

print(classification_report(
    y_test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    y_pred_binary,
    target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

))

               precision    recall  f1-score   support

        toxic       0.41      0.91      0.57      6090
 severe_toxic       0.11      0.91      0.19       367
      obscene       0.42      0.89      0.57      3691
       threat       0.15      0.82      0.25       211
       insult       0.35      0.89      0.50      3427
identity_hate       0.17      0.86      0.28       712

    micro avg       0.34      0.90      0.50     14498
    macro avg       0.27      0.88      0.39     14498
 weighted avg       0.38      0.90      0.52     14498
  samples avg       0.06      0.09      0.07     14498



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/me

In [ ]:
with open('classification_report_logistic.json', 'w') as f:
    json.dump(report_dict_logistic, f, indent=4)